In [1]:
import numpy as np
from collections import defaultdict

In [2]:
"""
Hit Rate

Calculates Hit Rate score from a set of recommendations.  
For each positive edge (purchase, review, etc.) in the test set, we generate a list of recommendations. Hit-rate is 
defined as the fraction of edges in the test set that is reflected in the first K recommendations for that user. 
This metric directly measures the probability that recommendations
made by the algorithm contain the items related to the user u.

Parameters
----------
test_graph : DGL Heterograph 
    The test graph should contain two node types only.
recommendations : dictionary
    Dictionary of recommendations, in which keys corresponds to user indexes, and values corresponds to product indexes.
    This dictionary should be ordered by best to worst recommendations for each user key. 
edge_label: string
    Name of the edge type (eg: purchase, review, etc.)
K: int
    Number of recommendations that will be considered for the evaluation.  

Returns: float
    Hit Rate Score
-------
DGL Heterograph"""
    
def hit_rate(test_graph, recommendations, edge_label, K):
    
    edges = torch.stack((test_graph.edges(etype=edge_label))).numpy().T
    edges_dict = defaultdict(list)
    
    for x,y in edges:
        edges_dict[x].append(y)
    
    hits, total = 0, 0
    for k, v in edges_dict.items():
        hits += sum(edge in v for edge in recommendations.get(k))
        total += len(v)

    return hits/total
    

In [5]:
"""
Mean Reciprocal Rank

Score that takes into account of the rank of each item among user's recommendations. 

Parameters
----------
test_graph : DGL Heterograph 
    The test graph should contain two node types only.
recommendations : dictionary
    Dictionary of recommendations, in which keys corresponds to user indexes, and values corresponds to product indexes.
    This dictionary should be ordered by best to worst recommendations for each user key. 
edge_label: string
    Name of the edge type (eg: purchase, review, etc.)
scaling_factor: int, optional
    For bigger datasets, the scaling factor ensures that, for example, the difference between rank 
    at 1,000 and rank at 2,000 is still noticeable, instead of being very close to 0.

Returns: float
    MMR Score
-------
DGL Heterograph"""
    
def mmr(test_graph, recommendations, edge_label, scaling_factor = 1):
    
    edges = torch.stack((test_graph.edges(etype=edge_label))).numpy().T
    edges_dict = defaultdict(list)
    
    for x,y in edges:
        edges_dict[x].append(y)
    
    agg_rec_rank, total = 0, 0
    for user, product_list in edges_dict.items():
        for product in product_list:
            rec_rank = 0
            if product in recommendations.get(user):
                rec_rank = 1/((recommendations.get(user).index(product)+1)/scaling_factor)
            else:
                rec_rank = 1/(len(recommendations.get(k))+1)
            agg_rec_rank += rec_rank
            total += 1

    return agg_rec_rank/total
    